In [3]:
from pathlib import Path
import requests
import pandas as pd
import ipysheet

### Non-Commercial (Hedge Funds, Banks & Institutions)  
### Commercial (Corporations Hedging)

In [4]:
df = pd.read_parquet('data/all_cot.parquet')
df

noncommercial_long  \
product                                            report_date                       
#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERCA... 2015-01-06                62178   
                                                   2015-01-13                67320   
                                                   2015-01-20                69672   
                                                   2015-01-27                68386   
                                                   2015-02-03                67464   
...                                                                            ...   
YIELD CURVE 30-2 - CHICAGO BOARD OF TRADE          1996-10-29                    0   
                                                   1996-11-05                    0   
                                                   1996-11-12                    0   
                                                   1996-11-19                    0   
                                                   1996-11-26                    0   

                                                                noncommercial_short  \
product                                            report_date                        
#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERCA... 2015-01-06                 86465   
                                                   2015-01-13                 96043   
                                                   2015-01-20                101341   
                                                   2015-01-27                 99776   
                                                   2015-02-03                103147   
...                                                                             ...   
YIELD CURVE 30-2 - CHICAGO BOARD OF TRADE          1996-10-29                    46   
                                                   1996-11-05                    41   
                                                   1996-11-12                    43   
                                                   1996-11-19                    40   
                                                   1996-11-26                    40   

                                                                commercial_long  \
product                                            report_date                    
#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERCA... 2015-01-06            190168   
                                                   2015-01-13            194294   
                                                   2015-01-20            198419   
                                                   2015-01-27            208750   
                                                   2015-02-03            200968   
...                                                                         ...   
YIELD CURVE 30-2 - CHICAGO BOARD OF TRADE          1996-10-29               160   
                                                   1996-11-05               162   
                                                   1996-11-12               172   
                                                   1996-11-19               172   
                                                   1996-11-26               182   

                                                                commercial_short  
product                                            report_date                    
#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERCA... 2015-01-06             166416  
                                                   2015-01-13             166134  
                                                   2015-01-20             167793  
                                                   2015-01-27             182828  
                                                   2015-02-03             171529  
...                                                                          ...  
YIELD CURVE 30-2 - CHICAGO BOARD OF TRADE          1996-10-29        

In [5]:
# df['noncommercial_total'] = df['noncommercial_long'] + df['noncommercial_short']
# df['noncommercial_net_positions'] = df['noncommercial_long'] - df['noncommercial_short']
# df['commercial_total'] = df['commercial_long'] + df['commercial_short']
# df['commercial_net_positions'] = df['commercial_long'] - df['commercial_short']
# df['total_difference'] = df['noncommercial_net_positions'] - df['commercial_net_positions']
# df['Pct Long'] 

In [6]:
def filter_future(future_name: str, cot_data: pd.DataFrame):
    """Filter single future from the cot dataframe."""
    return df.loc[future_name, :].sort_index(ascending=False)

def load_summary(df: pd.DataFrame):
    """Calculate summary statistics about data column."""
    stat = pd.DataFrame()
    stat["min"] = df.min()
    stat["max"] = df.max()
    stat["rolling"] = df.head(12).mean().round()
    return stat

def add_features(long_position: pd.Series, short_position: pd.Series) -> pd.DataFrame:
    """Calculates features required for the report."""
    df = pd.merge(long_position, short_position, left_index=True, right_index=True)
    df['total'] = long_position + short_position
    df['net_positions'] = long_position - short_position
    df['pct_long'] = round(long_position / df['total'], 2)
    df['pct_short'] = round(short_position / df['total'], 2) 
    return df

In [7]:
gold = filter_future(
    future_name='GOLD - COMMODITY EXCHANGE INC.',
    cot_data=df)
gold.head()

,noncommercial_long,noncommercial_short,commercial_long,commercial_short
report_date,,,,
2023-10-31,261053,97628,105971,288678
2023-10-24,251469,102084,106180,272053
2023-10-17,231412,118674,110062,238061
2023-10-10,220332,148899,121341,211769
2023-10-03,228406,137180,110666,222648


In [8]:
ipysheet.from_dataframe(gold)

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0[.]0', row_end=460, row_start=0, squeeze_row=…

In [87]:
noncommercial_data = add_features(long_position=gold['noncommercial_long'], short_position=gold['noncommercial_short'])  
commercial_data = add_features(long_position=gold['commercial_long'], short_position=gold['commercial_short'])

,commercial_long,commercial_short,total,net_positions,pct_long,pct_short
report_date,,,,,,
2023-10-31,105971,288678,394649,-182707,0.27,0.73
2023-10-24,106180,272053,378233,-165873,0.28,0.72
2023-10-17,110062,238061,348123,-127999,0.32,0.68
2023-10-10,121341,211769,333110,-90428,0.36,0.64
2023-10-03,110666,222648,333314,-111982,0.33,0.67
...,...,...,...,...,...,...
2015-02-03,117174,320447,437621,-203273,0.27,0.73
2015-01-27,117326,323486,440812,-206160,0.27,0.73
2015-01-20,120946,298756,419702,-177810,0.29,0.71


In [73]:
load_summary(gold)

,min,max,rolling
noncommercial_long,150229,408349,236077.0
noncommercial_short,24653,222210,115460.0
commercial_long,86143,221358,112620.0
commercial_short,148443,571177,252067.0
noncommercial_total,242661,463217,351537.0
noncommercial_net_positions,-38175,353649,120617.0
commercial_total,290519,792535,364686.0
ommercial_net_positions,-385612,25866,-139447.0
commercial_net_positions,-385612,25866,-139447.0


In [61]:
gold['noncommercial_long'].min()

150229

In [16]:
futures = df.index.get_level_values(0).unique()
futures[futures.str.contains("GOLD")]

Index(['COMMODITY INDEX, GOLDMAN-SACHS - INTERNATIONAL MONETARY MARKET ',
       'GOLD - COMMODITY EXCHANGE INC.', 'GOLD - COMMODITY EXCHANGE INC. ',
       'GOLD - INTERNATIONAL MONETARY MARKET ',
       'GOLD, 100 TROY OZ - CHICAGO BOARD OF TRADE ',
       'GOLDMAN-SACHS COMMODITY INDEX - CHICAGO MERCANTILE EXCHANGE ',
       'GOLDMAN-SACHS COMMODITY INDEX - INTERNATIONAL MONETARY MARKET ',
       'MICRO GOLD - COMMODITY EXCHANGE INC.'],
      dtype='object', name='product')

In [24]:
df.loc['GOLD - COMMODITY EXCHANGE INC.', :].sort_index(ascending=False).head()

,noncommercial_long,noncommercial_short,commercial_long,commercial_short
report_date,,,,
2023-10-31,261053,97628,105971,288678
2023-10-24,251469,102084,106180,272053
2023-10-17,231412,118674,110062,238061
2023-10-10,220332,148899,121341,211769
2023-10-03,228406,137180,110666,222648


In [9]:
class Future:
    def __init__(self, data):
        self.data = data

    def table(self, styled=False, rows=None):
        df = self.data if rows == None else self.data[:rows]

        if styled == True:
            return self.style_data(df)
        else:
            return df

    def style_data(self):
        cmap = sns.diverging_palette(10, 240, n=9, as_cmap=True)
        return self.data.style.format(
            {
                "% Long": "{:.0%}",
                "% Short": "{:.0%}",
                "Long NC": "{:,}",
                "Short NC": "{:,}",
                "Total": "{:,}",
                "Net Pos": "{:,}",
            }
        ).background_gradient(cmap)

    def load_summary(self):
        stat = pd.DataFrame()
        stat["min"] = self.data.min()
        stat["max"] = self.data.max()
        stat["rolling"] = self.data.head(12).mean().round()
        return stat.T.style.format(
            {
                "Long NC": "{:,}",
                "Short NC": "{:,}",
                "Total": "{:,}",
                "% Long": "{:.0%}",
                "% Short": "{:.0%}",
                "Net Pos": "{:,}",
            }
        )